In [1]:
import json
import os
import numpy as np

In [2]:
def import_metrics_json(models_path, run_num):
    """
    Looks in {models_path}/{run_num}/metrics.json and returns the contents as a
    Python dictionary. Returns None if the path does not exist.
    """
    path = os.path.join(models_path, str(run_num), "metrics.json")
    if not os.path.exists(path):
        return None
    with open(path, "r") as f:
        return json.load(f)

In [3]:
def get_best_metric(models_path, metric_extract_func, metric_compare_func):
    """
    Given the path to a set of runs, determines the run with the best metric value,
    where the metric value is fetched by `metric_extract_func`. This function must
    take the imported metrics JSON and return the (scalar) value to use for
    comparison. The best metric value is determiend by `metric_compare_func`, which
    must take in two arguments, and return whether or not the _first_ one is better.
    Returns the number of the run, the value associated with that run, and a dict of
    all the values used for comparison.
    """
    # Get the metrics, ignoring empty or nonexistent metrics.json files
    metrics = {run_num : import_metrics_json(models_path, run_num) for run_num in os.listdir(models_path)}
    metrics = {key : val for key, val in metrics.items() if val}  # Remove empties
    
    # Get the best value
    best_run, best_val, all_vals = None, None, {}
    for run_num in metrics.keys():
        try:
            val = metric_extract_func(metrics[run_num])
        except Exception:
            print("Warning: Was not able to extract metric for run %s" % run_num)
            continue
        all_vals[run_num] = val
        if best_val is None or metric_compare_func(val, best_val):
            best_val, best_run = val, run_num
    return best_run, best_val, all_vals

In [4]:
models_path = "/users/amtseng/att_priors/models/trained_models/profile_models/SPI1_prior_savedata_gradabs/"
best_run, best_val, all_vals = get_best_metric(
    models_path,
    lambda metrics: np.mean(metrics["summit_prof_corr_losses"]["values"]),
    lambda x, y: x < y
)
print("Best run: %s" % best_run)
print("Associated value: %s" % best_val)
for key in sorted(all_vals.keys(), key=lambda x: int(x)):
    print(key, all_vals[key])

Best run: 25
Associated value: 201.23614661308147
1 226.36899854799196
2 226.30391280538535
3 226.22971023101152
4 226.35281953279554
5 226.26141019501912
6 226.25596731619774
7 226.26553161435345
8 226.2545102013937
9 226.26304027148996
10 226.31571769714355
11 226.290281580547
12 226.450161585992
14 218.1957363996383
15 226.25977567746403
16 226.25721335733743
17 226.2721361806976
18 226.26501096076024
19 203.23399234493402
21 226.26376073572996
22 220.9367990053302
23 226.2398168648432
25 201.23614661308147
26 226.27806446964578
27 226.269721755654
